<a href="https://colab.research.google.com/github/profteachkids/CHE3022_Fall2022/blob/main/NetworkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.optimize import root

In [2]:
mu = 1e-3  #Pa s
rho = 1e3   # kg/m3
g=9.81      #m/s2

In [3]:
def churchill(Re, eD):
    t1 = (-2.457*np.log((7./Re)**0.9 + 0.27*eD))**16
    t2 = (37530/Re)**16
    return 2* ((8/Re)**12 + 1/(t1+t2)**1.5)**(1/12.)

In [4]:
def headDrop(Q, L, D, eD):
   v=4*Q/(np.pi*(D**2))
   Re = rho * v * D / mu
   f = churchill(Re, eD)
   return 2*f*L*(v**2)/(g*D)

In [17]:
#hD is the decrease in head from inlet to outlet
#Positive hD indicates positive from inlet to outlet
def flow(hD, L, D, eD):

    # guess f = 0.01
    Qguess = np.pi*(D**2)/4 * np.sqrt(hD*g*D/(2*0.01*L))
    def eq(x):
        Q=x[0]
        return [hD - headDrop(Q, L, D, eD)]

    return root(eq, [Qguess]).x[0]

In [18]:
headDrop(0.1, 10, 0.1, 1e-6)

9.240640333391058

In [19]:
flow(9.24, 10, 0.1, 1e-6)

0.09999622779308243